In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/P_FINAL/entrega

/content/drive/.shortcut-targets-by-id/1HF7BrRwJ4feKbgtGzRUwuiebcM_-b8Js/P_FINAL/entrega


### Carga y preprocesamiento

In [ ]:
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np


CPU = False

def transform_data(x_train, x_test, img_size=(64,64), dtype=np.float32):
    x_train = np.asarray( [cv.resize(img, img_size) for img in x_train], np.float64 )
    x_test = np.asarray( [cv.resize(img, img_size) for img in x_test], np.float64 )
    mu = np.mean(x_train, axis=(0,1,2))
    sigma = np.std(x_train, axis=(0,1,2))
    x_train -= mu
    x_train /= sigma
    x_test -= mu
    x_test /= sigma
    x_train = x_train.transpose((0,3,1,2)).astype(dtype)
    x_test = x_test.transpose((0,3,1,2)).astype(dtype)
    return x_train, x_test

In [ ]:
data = tf.keras.datasets.cifar100.load_data(label_mode="fine")

In [ ]:
x_train = data[0][0][:,:,:,::-1]
y_train = data[0][1]
x_test = data[1][0][:,:,:,::-1]
y_test = data[1][1]

In [ ]:
x_train, x_test = transform_data(x_train, x_test)

In [ ]:
from sklearn.model_selection import train_test_split

batch_size = 128
x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                  y_train, 
                                                  test_size=0.15, 
                                                  shuffle=True, 
                                                  random_state=57)

### Instalar Caffe

In [ ]:
if CPU:
    !sudo apt install caffe-cpu
else:
    !sudo apt install caffe-cuda

In [ ]:
import caffe

if not CPU:
    caffe.set_mode_gpu()
    caffe.set_device(0)

### Entrenamiento última capa

In [ ]:
def step_over_chunks(solver, data_chunks, labels_chunks, img_shape):
    for dchunk, lchunk in zip(data_chunks, labels_chunks):
        new_shape = (len(dchunk),) + img_shape
        if solver.net.blobs['data'].data.shape != new_shape:
            solver.net.blobs['data'].reshape(*new_shape)
            solver.net.blobs['label'].reshape(len(dchunk))
        for i in range(len(dchunk)):
            solver.net.blobs['data'].data[i] = dchunk[i]
            solver.net.blobs['label'].data[i] = lchunk[i]
        solver.step(1)

def test_net(test_net, x_test, y_test, batch_size):
    test_data_chunks = [ x_test[x:x+batch_size] for x in range(0, len(x_test), batch_size) ]
    test_labels_chunks = [ np.squeeze(y_test[y:y+batch_size]) for y in range(0, len(y_test), batch_size) ]
    loss_sum = 0
    accuracy = 0

    for dchunk, lchunk in zip(test_data_chunks, test_labels_chunks):
        new_shape = (len(dchunk),) + x_test[0].shape
        if test_net.blobs['data'].data.shape != new_shape:
            test_net.blobs['data'].reshape(*new_shape)
            test_net.blobs['label'].reshape(len(dchunk))
        test_net.blobs['data'].data[:] = dchunk[:]
        test_net.blobs['label'].data[:] = lchunk[:]

        test_net.forward()
        loss_sum += test_net.blobs['loss'].data
        accuracy += test_net.blobs['accuracy'].data

    return loss_sum/len(test_data_chunks), accuracy/len(test_data_chunks)

def fit(solver, x_train, y_train, x_val, y_val, niter, batch_size, patience=5, epsilon=0.0001):
    train_loss, val_loss = np.zeros(niter), np.zeros(niter)

    data_chunks = [ x_train[x:x+batch_size] for x in range(0, len(x_train), batch_size) ]
    labels_chunks = [ y_train[y:y+batch_size] for y in range(0, len(y_train), batch_size) ]

    final_it = niter-1
    best_loss = float("inf")
    worse_loss_counter = 0

    for it in range(niter):
        step_over_chunks(solver, data_chunks, labels_chunks, x_train[0].shape)
        train_loss[it] = solver.net.blobs['loss'].data
        val_loss[it], _ = test_net(solver.test_nets[0], x_val, y_val, batch_size)
        print('iter {}, loss={}, val_loss={}'.format(it, train_loss[it], val_loss[it]))

        if val_loss[it] < best_loss - epsilon:
            best_loss = val_loss[it]
            worse_loss_counter = 0
        else:
            worse_loss_counter += 1
            if worse_loss_counter > patience:
                final_it = it
                break
    
    return train_loss[0:final_it+1], val_loss[0:final_it+1]

In [ ]:
# Entrenamiento última capa

solver_cifar100 = caffe.get_solver('./model_cifar100/solver_cifar100.prototxt')
solver_cifar100.net.copy_from('./model_cfn_jps/cfn_jps.caffemodel')

train_loss, val_loss = fit(solver_cifar100, x_train, y_train, x_val, y_val, 200, batch_size)

In [ ]:
# Comprobar la precisión del modelo final

_, accuracy = test_net(solver_cifar100.test_nets[0], x_test, y_test, batch_size)
print("Precisión en test: {}".format(accuracy))

In [ ]:
# Guardar los pesos del modelo

solver_cifar100.net.save('./model_cifar100/cifar100.caffemodel')

In [ ]:
# Gráfica train-loss, val-loss

_, ax1 = plt.subplots()
ax1.plot(np.arange(len(train_loss)), train_loss, label='Train loss')
ax1.plot(np.arange(len(val_loss)), val_loss, label='Validation loss')
ax1.set_xlabel('Epochs')
ax1.legend(loc='best')

### Ajuste fino

In [ ]:
# Fine Tuning

solver_cifar100_ft = caffe.get_solver('./model_cifar100/solver_cifar100_fine_tuning.prototxt')
solver_cifar100_ft.net.copy_from('./model_cifar100/cifar100.caffemodel')

train_loss_ft, val_loss_ft = fit(solver_cifar100_ft, x_train, y_train, x_val, y_val, 50, batch_size, patience=1)

In [ ]:
# Comprobar la precisión del modelo final

_, accuracy = test_net(solver_cifar100_ft.test_nets[0], x_test, y_test, batch_size)
print("Precisión en test: {}".format(accuracy))

In [ ]:
# Guardar los pesos del modelo

solver_cifar100_ft.net.save('./cifar100_fine_tuning.caffemodel')

In [ ]:
# Gráfica train-loss, val-loss

_, ax1 = plt.subplots()
ax1.plot(np.arange(len(train_loss_ft)), train_loss_ft, label='Train loss')
ax1.plot(np.arange(len(val_loss_ft)), val_loss_ft, label='Validation loss')
ax1.set_xlabel('Epochs')
ax1.legend(loc='best')

### Filtros aprendidos

In [ ]:
def vis_square(data):
    """Take an array of shape (n, height, width) or (n, height, width, 3)
       and visualize each (height, width) thing in a grid of size approx. sqrt(n) by sqrt(n)"""
    
    # normalize data for display
    data = (data - data.min()) / (data.max() - data.min())
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = (((0, n ** 2 - data.shape[0]),
               (0, 1), (0, 1))                 # add some space between filters
               + ((0, 0),) * (data.ndim - 3))  # don't pad the last dimension (if there is one)
    data = np.pad(data, padding, mode='constant', constant_values=1)  # pad with ones (white)
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    plt.imshow(data, cmap='gray'); plt.axis('off')


[enlace de la función vis_square](https://github.com/arundasan91/Deep-Learning-with-Caffe/blob/master/Deep-Neural-Network-with-Caffe/Deep%20Neural%20Network%20with%20Caffe.md)

In [ ]:
# Cargar las dos versiones del modelo (solo con entrenamiento en la última capa y con fine tuning)

model_cifar100 = caffe.Net('./model_cifar100/model_cifar100.prototxt','./model_cifar100/cifar100.caffemodel',caffe.TEST)
model_cifar100_ft = caffe.Net('./model_cifar100/model_cifar100_fine_tuning.prototxt','./model_cifar100/cifar100_fine_tuning.caffemodel',caffe.TEST)

In [ ]:
# Obtener los filtros de la primera capa convolucional

filters = model_cifar100.params['conv1_s1'][0].data
vis_square(filters.transpose(0, 2, 3, 1))

In [ ]:
# Obtener los filtros de la primera capa convolucional

filters = model_cifar100_ft.params['conv1_s1'][0].data
vis_square(filters.transpose(0, 2, 3, 1))